#ローカルLLMのサンプル利用コード
##利用方法
・基本的には全てのセルを実行することで利用できます。

・cpuインスタンスのまま利用することを推奨します。
（GPUを利用した方が高速です）

（変更する際は，「ランタイム」→「ランタイムのタイプを変更」からGPUを選択してください）

##注意点
・本ファイルの相対的な場所は変更しないでください。

・本ファイルの名前を変更しないでください。

変更する場合は，[2]セル目のファイル名も一緒に変更してください。

・[1]セル目の実行時にセッションの再起動を求められることがあります。

その場合はポップアップに従って再起動したあと，全てのセルを再度実行してください。

In [ ]:
#ローカルLLM で必要なモジュールのインストール

!set LLAMA_CUBLAS=1
!set CMAKE_ARGS=-DLLAMA_CUBLAS=on
!set FORCE_CMAKE=1
!python -m pip install llama-cpp-python==0.2.77 --prefer-binary  --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu122
!pip install torch==2.3.0 torchvision==0.18.0 torchaudio==2.3.0 --index-url https://download.pytorch.org/whl/cu121

下記のコードを実行すると，認証が求められるので，認証してください。（Google colabにマイドライブの中を読めるようにする認証です）

In [1]:
#Google Driveのフォルダをマウント（認証入る）
from google.colab import drive
drive.mount('/content/drive')


# カレントディレクトリを本ファイルが存在するディレクトリに変更する。
import glob
import os
pwd = os.path.dirname(glob.glob('/content/drive/MyDrive/**/colab_LocalLLM_sample/LocalLLM_sample.ipynb', recursive=True)[0])
print(pwd)

%cd $pwd
!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/docomo/colab_zenn/colab_LocalLLM_sample
/content/drive/MyDrive/docomo/colab_zenn/colab_LocalLLM_sample
/content/drive/MyDrive/docomo/colab_zenn/colab_LocalLLM_sample


In [2]:
#LocalLLMを切り出したモジュールをimportする
from module.module_LocalLLM import LLM

モデルを変更する場合は，下記のconfigを変更してください。

「;」を付けるとコメントアウトできます。


In [3]:
#モデルの設定を行う。

config_text = """
[DEFAULT]
ai_agent = ai
agent_dir = Agents

[LLM]
llm_model = ./model_assets/LocalLLM/swallow-13b-instruct.Q4_K_M.gguf
;llm_model = TheBloke/swallow-13b-instruct.Q4_K_M.gguf
;n_gpu_layers_num = 0
n_gpu_layers_num = -1
n_ctx = 2048
max_tokens = 1024
temperature = 1.0
system_prompt_first_file_path = ./prompt/LLMsysFirstPrompt.txt
system_prompt_file_path = llm_agent.txt
system_prompt_end_file_path = ./prompt/LLMsysEndPrompt.txt
"""

with open("configs/config.ini", "w", encoding="utf-8") as f:
  f.write(config_text)

下記のプロンプトの内容をLLMに入力します。

(sys_first_prompt+sys_prompt+sys_end_promptがシステムプロンプトの内容になります。）


In [4]:
#入力するプロンプトを設定する。

#デフォルトのシステムプロンプトを利用しない場合の、システムプロンプトを記述する
sys_temp_prompt = """あなたは関西人です。関西弁でuserに対して返答をしてください。"""

#LLMに質問する内容を記載する。
user_prompt = """こんにちは！
はじめまして。
あなたの名前を教えてください。"""


In [6]:
#デフォルトのシステムプロンプトを利用する場合
llm = LLM()
messages = llm.simpleLLM(user_prompt, temp_sys_prompt = None)

print(messages)

llama_model_loader: loaded meta data with 22 key-value pairs and 363 tensors from ./model_assets/LocalLLM/swallow-13b-instruct.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.at

もちろんです。
私の名前はあいです。よろしくね。



In [ ]:
#上のセルで指定したシステムプロンプトを利用する場合
llm_temp = LLM()
messages = llm_temp.simpleLLM(user_prompt, temp_sys_prompt = sys_temp_prompt)

print(messages)

続いて、stream処理を実装しています。
（Local LLMは処理するシステムによっては、すべてのテキストを出力するまでに時間がかかるため）

一文字ずつ出力させるために，print関数を使っているが，煩わしかったら消してください。


2回目に実行する場合（インスタンスが保持されている状態）は，インスタンスを再起動してから，[2]セル目から実行してください。

In [5]:
llm_stream = LLM()
prompt, max_tokens, temperature = llm_stream.simpleLLMstream_prepare(user_prompt)

messages = llm_stream.llm.create_completion(
            prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            stop = ["user","ユーザ:"],
            stream=True)

all_text = ""
for chunk in messages:
    all_text += chunk['choices'][0]['text']
    print(chunk['choices'][0]['text'], end = "", flush = True)

print("finish")
print(all_text)

llama_model_loader: loaded meta data with 22 key-value pairs and 363 tensors from ./model_assets/LocalLLM/swallow-13b-instruct.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.at

あいです。
よろしくお願いします!


llama_print_timings:        load time =    1878.62 ms
llama_print_timings:      sample time =       7.32 ms /    11 runs   (    0.67 ms per token,  1503.55 tokens per second)
llama_print_timings: prompt eval time =    2258.73 ms /   601 tokens (    3.76 ms per token,   266.08 tokens per second)
llama_print_timings:        eval time =     473.36 ms /    10 runs   (   47.34 ms per token,    21.13 tokens per second)
llama_print_timings:       total time =    2759.75 ms /   611 tokens


finish
あいです。
よろしくお願いします!
